Author: Dana Chermesh Reshef, DRAW Brooklyn<br>
April 2019

### _Digital CEQR -- #2_
# Data Source: Housing Permits from Census Bureau Building Permits Survey (BPS) 
retrieved from [Census Bureau Building Permits Survey](https://www.census.gov/construction/bps/), County and Place level

### _Example Approach:_
- Analyze permits issuance pre- and post-recession, meaning for 2000-2008 and for 2008-2017
- Analyze permits issuance pre- and post-an event, i.e adjuscent rezoning / big development

---- 
# 0 - Imports

In [122]:
import pandas as pd

# reading in my api key saved in censusAPI.py as
# myAPI = 'XXXXXXXXXXXXXXX'
# request an api key in: https://api.census.gov/data/key_signup.html
from censusAPI import myAPI

# 1 - Data

## 1.1 Total housing units 2000 (base point); _Decennial Census 2000, SF1_
Data were retrieved using my census API (_**see explanation in README.md of this repo or in former notebooks**_)
- [variables](https://api.census.gov/data/2000/sf1/variables.html)

In [123]:
# total POP and total housing units for all counties in the US, 2010
# P001001 = total pop
# H001001 = total housing units

housing00 = pd.read_json('https://api.census.gov/data/2000/sf1?get=P001001,H001001'+
                         '&for=county:*&in=state:*&key='+myAPI)
housing00.columns = housing00.iloc[0]
housing00 = housing00[1:]

housing00.columns = ['pop2000', 'hu2000', 'state', 'county']
housing00['STCO'] = housing00[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

print(housing00.shape)
housing00.head()

(3141, 5)


,pop2000,hu2000,state,county,STCO
1,43671,17662,01,001,01001
2,140415,74285,01,003,01003
3,29038,12461,01,005,01005
4,20826,8345,01,007,01007
5,51024,21158,01,009,01009


## Total housing units 2008 (base point); _PEP housing 2000-2008_ 
** Data need to be downloaded to your local machine, then read in 2008 estimates only 
- Source: [Housing Units Intercensal Datasets: 2000-2010](https://www.census.gov/data/datasets/time-series/demo/popest/intercensal-2000-2010-housing-units.html)
- [Direct download](https://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/housing/hu-est00int-tot.csv)

In [124]:
housing08 = pd.read_excel('../rp-USmetros_comparison/data/hu-est00int-tot.xlsx')
housing08 = housing08[housing08['SUMLEV'] != 40]
housing08 = housing08.drop(['HUESTBASE2000', 'HUEST_2000', 'HUEST_2001', 'HUEST_2002',
                            'HUEST_2003', 'HUEST_2004', 'HUEST_2005', 'HUEST_2006',
                            'HUEST_2007', 'HUEST_2009', 'HUCENSUS2010', 'HUEST_2010',
                            'SUMLEV'], axis=1)

housing08.columns = ['state', 'county', 'NAME', 'hu2008']
housing08['state'] = housing08['state'].apply(lambda x: '{0:0>2}'.format(x))
housing08['county'] = housing08['county'].astype(int).apply(lambda x: '{0:0>3}'.format(x))
housing08['STCO'] = housing08[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

housing08 = housing08.drop(['state', 'county'], axis=1)

print(housing08.shape)
housing08.head()

(3143, 3)


,NAME,hu2008,STCO
1,Autauga County,21605,01001
2,Baldwin County,102011,01003
3,Barbour County,12009,01005
4,Bibb County,8863,01007
5,Blount County,23500,01009


### Places

----

# 1.2 Perimts issuance data from 2008 to 2017 - [Building Permits Survey](https://www.census.gov/construction/bps/)
The data were retrieved from the Census Bureau [Building Permits Survey](https://www.census.gov/construction/bps/), [Permits by County or Place](http://www2.census.gov/econ/bps).

For downloading the data, please go to the [County/](https://www2.census.gov/econ/bps/County/) page or the [Places/](https://www2.census.gov/econ/bps/Place/) page, and choose:
- [co2017a.txt](https://www2.census.gov/econ/bps/County/co2017a.txt) 
- [co2016a.txt](https://www2.census.gov/econ/bps/County/co2016a.txt) 
- [co2015a.txt](https://www2.census.gov/econ/bps/County/co2015a.txt) 
- [co2014a.txt](https://www2.census.gov/econ/bps/County/co2014a.txt) 
- [co2013a.txt](https://www2.census.gov/econ/bps/County/co2013a.txt) 
- [co2012a.txt](https://www2.census.gov/econ/bps/County/co2012a.txt) 
- [co2011a.txt](https://www2.census.gov/econ/bps/County/co2011a.txt) 
- [co2010a.txt](https://www2.census.gov/econ/bps/County/co2010a.txt) 

and:
- [ne2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [mw2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [so2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [we2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)<br>
.<br>
.<br>
.<br>
.<br>

Data can be read directly to this notebook using pandas `read_table`, as bellow.

## 2008-2017 _(including 2008)_ Permits 

In [125]:
tablesCO = ['co2017a', 'co2016a', 'co2015a', 'co2014a',
            'co2013a', 'co2012a', 'co2011a', 'co2010a',
            'co2009a', 'co2008a']

COdata08 = []

for year in tablesCO:
    
    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units'] = df['1unit'] + df['2unit']
    df['+3units'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata08.append(df)

COall08_17 = pd.concat(COdata08).groupby('STCO').sum()
print(COall08_17.shape)
print(COall08_17.dtypes)
COall08_17.head()

data: co2017a (3038, 5)
data: co2016a (3039, 5)
data: co2015a (3044, 5)
data: co2014a (3038, 5)
data: co2013a (3027, 5)
data: co2012a (3026, 5)
data: co2011a (3026, 5)
data: co2010a (3026, 5)
data: co2009a (3026, 5)
data: co2008a (3026, 5)
(3061, 2)
1-2units    int64
+3units     int64
dtype: object


,1-2units,+3units
STCO,,
01001,1419,312
01003,13407,2229
01005,81,40
01007,140,42
01009,107,40


## 2000-2008 _(including 2000, excluding 2008)_ Permits

In [126]:
tablesCO00 = ['co2007a', 'co2006a', 'co2005a', 'co2004a',
              'co2003a', 'co2002a', 'co2001a', 'co2000a']

COdata00 = []

for year in tablesCO00:

    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units_00'] = df['1unit'] + df['2unit']
    df['+3units_00'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata00.append(df)

COall00_08 = pd.concat(COdata00).groupby('STCO').sum()
print(COall00_08.shape)
print(COall00_08.dtypes)
COall00_08.head()

data: co2007a (3026, 5)
data: co2006a (3026, 5)
data: co2005a (3027, 5)
data: co2004a (3027, 5)
data: co2003a (3021, 5)
data: co2002a (3013, 5)
data: co2001a (3014, 5)
data: co2000a (3018, 5)
(3050, 2)
1-2units_00    int64
+3units_00     int64
dtype: object


,1-2units_00,+3units_00
STCO,,
01001,2203,224
01003,21082,9032
01005,180,40
01007,42,0
01009,389,91


## 1.4 Merging datasets

### merging 00-08 with 08-17 datasets

In [133]:
permits = COall00_08.merge(COall08_17, on='STCO')

print(permits.shape)
permits.head()

(3029, 4)


,1-2units_00,+3units_00,1-2units,+3units
STCO,,,,
01001,2203,224,1419,312
01003,21082,9032,13407,2229
01005,180,40,81,40
01007,42,0,140,42
01009,389,91,107,40


In [113]:
type(permits.index[0])

str

-----

# PLACES 
Major Cities within the Regions

### _Note#1: PLACES in the housing permits survey are separated to Midwest, Northeast, South and West Regions; data were downloaded through each of these for years 2010 - 2017 and were concatenated, then cleaned to include our targeted cities only_

### _Note#2: NYC is considered as 5 places, which are its 5 boroughs_

### _Note#3: There were changes in GEOID's for several places_

## BPS data acquisition for places
## 2008-2017 _(including 2008)_ Permits 

In [22]:
# all needed files to be read in, aggregated by area
Northeast = ['ne2017a', 'ne2016a', 'ne2015a', 'ne2014a','ne2013a', 
             'ne2012a', 'ne2011a', 'ne2010a', 'ne2009a', 'ne2008a']
Midwest = ['mw2017a', 'mw2016a', 'mw2015a', 'mw2014a','mw2013a', 
           'mw2012a', 'mw2011a', 'mw2010a', 'mw2009a', 'mw2008a']
South = ['so2017a', 'so2016a', 'so2015a', 'so2014a','so2013a', 
         'so2012a', 'so2011a', 'so2010a', 'so2009a', 'so2008a']
West = ['we2017a', 'we2016a', 'we2015a', 'we2014a','we2013a', 
        'we2012a', 'we2011a', 'we2010a', 'we2009a', 'we2008a']

# all US areas
US_AREAS = {'Northeast' : Northeast,
            'Midwest' : Midwest,
            'South' : South,
            'West' : West}

# empty list for all dataframes to be added
USA = []

# reading data into python pandas
for area in US_AREAS:
    for year in US_AREAS[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Pop', 'Footnote', 'Census Place',
                      '6-Digit', 'FIPS MCD', 'Number of',
                      'Zip', 'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units'] = df['1unit'] + df['2unit']
        df['+3units'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['Place'] = df['Place'].apply(lambda x: '{0:0>5}'.format(x))
        df['GEOID'] = df[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', 'Place'], axis=1)

        df['1-2units'] = df['1-2units'].astype(int)
        df['+3units'] = df['+3units'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA.append(df)


hp_Places = pd.concat(USA).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
hp_Places = hp_Places[11:]

print(hp_Places.shape)
print(hp_Places.dtypes)
hp_Places.head(15)

mw2017a
mw2016a
mw2015a
mw2014a
mw2013a
mw2012a
mw2011a
mw2010a
mw2009a
mw2008a
we2017a
we2016a
we2015a
we2014a
we2013a
we2012a
we2011a
we2010a
we2009a
we2008a
so2017a
so2016a
so2015a
so2014a
so2013a
so2012a
so2011a
so2010a
so2009a
so2008a
ne2017a
ne2016a
ne2015a
ne2014a
ne2013a
ne2012a
ne2011a
ne2010a
ne2009a
ne2008a
(30152, 5)
GEOID       object
CSA         object
Name        object
1-2units     int64
+3units      int64
dtype: object


,GEOID,CSA,Name,1-2units,+3units
11,0100124,222,Abbeville,5,0
12,0100460,142,Adamsville,10,0
13,0100676,999,Akron town,0,0
14,0100820,142,Alabaster,877,0
15,0100988,999,Albertville,100,8
16,0101132,999,Alexander City,130,56
17,0101228,999,Aliceville,15,0
18,0101708,999,Andalusia,81,0
19,0101852,999,Anniston,26,0
20,0102116,999,Arab,122,0


## 2000-2008 _(including 2000, excluding 2008)_ Permits 
### ** _at 2007 the data were changes as two columns were added; the data for 2007 and for 2000-2006 (included) were obtained separately_

In [68]:
# all needed files to be read in, aggregated by area
Northeast07 = ['ne2007a']
Midwest07 = ['mw2007a']
South07 = ['so2007a']
West07 = ['we2007a']

# all US areas
US_AREAS07 = {'Northeast' : Northeast07,
            'Midwest' : Midwest07,
            'South' : South07,
            'West' : West07}

# empty list for all dataframes to be added
USA00 = []

# reading data into python pandas
for area in US_AREAS07:
    for year in US_AREAS07[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Pop', 'Footnote', 'Census Place',
                      'Place', 'FIPS MCD', 'Number of',
                      'Zip', 'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units00'] = df['1unit'] + df['2unit']
        df['+3units00'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['6-Digit'] = df['6-Digit'].apply(lambda x: '{0:0>6}'.format(x))
        df['GEOID'] = df[['State', '6-Digit']].apply(lambda x: ''.join(x), axis=1)

        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', '6-Digit'], axis=1)

        df['1-2units00'] = df['1-2units00'].astype(int)
        df['+3units00'] = df['+3units00'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA00.append(df)


# hp_Places07 = pd.concat(USA07).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
# hp_Places07 = hp_Places07[11:]

# print(hp_Places07.shape)
# print(hp_Places07.dtypes)
# hp_Places07.head(15)

mw2007a
we2007a
so2007a
ne2007a


In [69]:
# all needed files to be read in, aggregated by area
Northeast00 = ['ne2006a', 'ne2005a', 'ne2004a', 
               'ne2003a', 'ne2002a', 'ne2001a', 'ne2000a']
Midwest00 = ['mw2006a', 'mw2005a', 'mw2004a', 
             'mw2003a', 'mw2002a', 'mw2001a', 'mw2000a']
South00 = ['so2006a', 'so2005a', 'so2004a', 
           'so2003a', 'so2002a', 'so2001a', 'so2000a']
West00 = ['we2006a', 'we2005a', 'we2004a',
          'we2003a', 'we2002a', 'we2001a', 'we2000a']

# all US areas
US_AREAS00 = {'Northeast' : Northeast00,
            'Midwest' : Midwest00,
            'South' : South00,
            'West' : West00}

# empty list for all dataframes to be added
USA00 = []

# reading data into python pandas
for area in US_AREAS00:
    for year in US_AREAS00[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Place', 
                      'CSA', 'CBSA', 'Footnote', 'Central', 
                      'Zip','Region', 'Division', 'Number of',
                      'Place2','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Footnote', 'Place2',
                      'Place', 'Number of', 'Zip',
                      'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units00'] = df['1unit'] + df['2unit']
        df['+3units00'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['6-Digit'] = df['6-Digit'].apply(lambda x: '{0:0>6}'.format(x))
        df['GEOID'] = df[['State', '6-Digit']].apply(lambda x: ''.join(x), axis=1)
        
        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', '6-Digit'], axis=1)

        df['1-2units00'] = df['1-2units00'].astype(int)
        df['+3units00'] = df['+3units00'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA00.append(df)


hp_Places00 = pd.concat(USA00).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
hp_Places00 = hp_Places00[11:]

print(hp_Places00.shape)
print(hp_Places00.dtypes)
hp_Places00.head(15)

mw2006a
mw2005a
mw2004a
mw2003a
mw2002a
mw2001a
mw2000a
we2006a
we2005a
we2004a
we2003a
we2002a
we2001a
we2000a
so2006a
so2005a
so2004a
so2003a
so2002a
so2001a
so2000a
ne2006a
ne2005a
ne2004a
ne2003a
ne2002a
ne2001a
ne2000a
(40940, 5)
GEOID         object
CSA           object
Name          object
1-2units00     int64
+3units00      int64
dtype: object


,GEOID,CSA,Name,1-2units00,+3units00
11,01008000,999,Aliceville,0,0
12,01008000,9999,Aliceville,0,3
13,01012000,999,Andalusia,0,82
14,01012000,9999,Andalusia,0,26
15,01014000,0450,Anniston,0,0
16,01014000,999,Anniston,0,43
17,01016000,999,Arab,4,109
18,01016000,9999,Arab,0,81
19,01017000,290,Ardmore town,0,29
20,01017000,3440,Ardmore town,0,6


## 2.2 Merging datasets
### Merging 00-08 with 08-17 datasets

In [72]:
# converting datatype to int
hp_Places['GEOID'] = hp_Places['GEOID'].apply(lambda x: '{0:0>8}'.format(x))
hp_Places['GEOID'] = hp_Places['GEOID'].astype(int)
hp_Places['CSA'] = hp_Places['CSA'].astype(int)

hp_Places00['GEOID'] = hp_Places00['GEOID'].astype(int)
hp_Places00['CSA'] = hp_Places00['CSA'].astype(int)

In [73]:
permits00 = hp_Places00.merge(hp_Places, on='GEOID')

print(permits00.shape)
permits00.head()

(318, 9)


,GEOID,CSA_x,Name_x,1-2units00,+3units00,CSA_y,Name_y,1-2units,+3units
0,1213000,2180,Gordon town,0,4,999,Clewiston,104,53
1,1213000,222,Gordon town,0,0,999,Clewiston,104,53
2,1224000,516,Gulf Shores,34,657,370,Fort Lauderdale,500,1409
3,1224000,516,Gulf Shores,34,657,999,Fort Lauderdale,409,1244
4,1224000,5160,Gulf Shores,36,673,370,Fort Lauderdale,500,1409
